# 導入函式庫

In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
import random as r
import math

# 對資料集內的各個資料夾取得圖片、進行降採樣、打亂資料，並且依據輸入比例分為訓練集和測試集，最後輸出圖片路徑和對應label list

In [2]:
def get_files(file_dir, train_size):

    images = []   
    subfolders = []
    train_array = []
    test_array = []
    train_img_list = []
    test_img_list = []
    train_all_img = []
    test_all_img = []
 
    for dirPath, dirNames, fileNames in os.walk(file_dir):
        
        names = []
        for name in fileNames:
            names.append(os.path.join(dirPath, name))

        for name in dirNames:
            subfolders.append(os.path.join(dirPath, name))

        r.shuffle(names)
        if names != []:
            images.append(names)

    mincount = float("Inf")
    for num_folder in subfolders:
        n_img = len(os.listdir(num_folder))
        
        if n_img < mincount:
            mincount = n_img

    for i in range(len(images)):
        images[i] = images[i][0:mincount]
        train_img_list.append([])
        test_img_list.append([])
        train_num = math.ceil(len(images[i]) * train_size)
        train_img_list[i] = images[i][0:train_num]
        test_img_list[i] = images[i][train_num:]

    for i in range(len(images)):
        train_all_img.extend(train_img_list[i])


    for i in range(len(images)):
        test_all_img.extend(test_img_list[i])
    
    train_labels = []
    test_labels = []
    for count in range(len(subfolders)):
        train_labels = np.append(train_labels, len(train_img_list[0]) * [count])

    for count in range(len(subfolders)):
        test_labels = np.append(test_labels, len(test_img_list[0]) * [count])

    train_array = np.array([train_all_img, train_labels])
    train_array = train_array[:, np.random.permutation(train_array.shape[1])].T
    test_array = np.array([test_all_img, test_labels])
    test_array = test_array[:, np.random.permutation(test_array.shape[1])].T
    
    train_img = list(train_array[:, 0])
    train_labels = list(train_array[:, 1])
    train_labels = [int(float(i)) for i in train_labels]
    
    test_img = list(test_array[:, 0])
    test_labels = list(test_array[:, 1])
    test_labels = [int(float(i)) for i in test_labels]
    

    return train_img, train_labels, test_img, test_labels

# 為了將各個圖片集轉為TFRecord檔，將每張圖片和其label轉為tf.train.Feature的格式

In [3]:
# 轉Int64資料為 tf.train.Feature 格式
def int64_feature(value):
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

# 轉Bytes資料為 tf.train.Feature 格式
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def convert_to_TFRecord(images, labels, name):
    
    filename = 'C:/SPLAB/TEST1/' + name + '.tfrecords'
    
    n_samples = len(labels)
    if np.shape(images)[0] != n_samples:
        raise ValueError('Images size {} does not match label size {}'.format(images.shape[0], n_samples))

    writer = tf.python_io.TFRecordWriter(filename)       # TFRecordWriter class
    print ('Convert to TFRecords...')
    for i in np.arange(0, n_samples):
        try:
            
            image = cv2.imread(images[i], 0)                # type(image) must be array
            image_raw = image.tostring()                  # transform array to bytes
            label = int(labels[i])

            ftrs = tf.train.Features(
                    feature={'Label': int64_feature(label),
                             'image_raw': bytes_feature(image_raw)}
                   )
    
            example = tf.train.Example(features=ftrs)
            writer.write(example.SerializeToString())
        except IOError as e:
            print ('Could not read:{}'.format(images[i]))
            print ('Skip it!')
    writer.close()
    print ('Done')

def main():
    train_dataset_dir = 'C:/SPLAB/TEST1/DATA1'
    train_img, train_labels, test_img, test_labels = get_files(train_dataset_dir,
                                                               train_size=0.8)
    
    
    # convert TFRecord file
    TFRecord_list = ['train', 'test']
    img_labels_list = [[train_img, train_labels], [test_img, test_labels]]

    for index, TFRecord_name in enumerate(TFRecord_list):
        convert_to_TFRecord(img_labels_list[index][0], img_labels_list[index][1],
                            TFRecord_name)
    #convert_to_TFRecord(images, labels, 'C:/SPLAB/TEST1/Train.tfrecords')
if __name__ == '__main__':
    main()


Convert to TFRecords...
Done
Convert to TFRecords...
Done


# 將前面已轉為TFRecord檔的訓練集進行讀取和解碼成圖片與其標籤

In [4]:
def read_and_decode(filename, batch_size, MAX_EPOCH): 
    # 建立文件名隊列
    filename_queue = tf.train.string_input_producer([filename], num_epochs = MAX_EPOCH)
    
    # 數據讀取器
    reader = tf.TFRecordReader()
    key, values = reader.read(filename_queue)    # filename_queue
    features = tf.parse_single_example(values,
                                       features={
                                            'Label': tf.FixedLenFeature([], tf.int64),
                                            'image_raw': tf.FixedLenFeature([], tf.string),
                                       })
        # decode to tf.uint8
    image = tf.decode_raw(features['image_raw'], tf.uint8)  # tf.uint8

        # reshape
    image = tf.reshape(image, [360, 210, 1])

        # label
    label = tf.cast(features['Label'], tf.int64)    # label tf.int32

    image_batch, label_batch =tf.train.shuffle_batch(
                                 [image, label],
                                 batch_size=batch_size,
                                 capacity=10000 + 3 * batch_size,
                                 min_after_dequeue=1000)

    return image_batch, label_batch

# 為了建立卷積網路，先將權重、偏差、卷積和池化定義出來

In [5]:
def Weight(shape, mean=0, stddev=1):
    init = tf.truncated_normal(shape, mean=mean, stddev=stddev)
    return tf.Variable(init)

def bias(shape, mean=0, stddev=1):
    init = tf.truncated_normal(shape, mean=mean, stddev=stddev)
    return tf.Variable(init)

def conv2d(x, W, strides=[1, 1, 1, 1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

def max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1]):
    return tf.nn.max_pool(x, ksize=ksize, strides=strides, padding='SAME')

# 進行卷積網路的建立，並且決定使用 ReLU 函數作為 Activate Function

# 再選擇合適的優化器、輸入學習率來進行迭代，並且定義如何計算正確率

In [6]:
filename = 'C:/SPLAB/TEST1/train.tfrecords'
# batch 可以自由設定
BATCH_SIZE = 64
MAX_EPOCH = 20
LABEL_NUM = 5

# Input images & labels
X  = tf.placeholder(tf.float32, shape = [None, 360, 210, 1])
y_ = tf.placeholder(tf.float32, shape = [None, LABEL_NUM])

# 卷積網路建立
# Conv1
W_conv1 = Weight([5, 5, 1, 16])
b_conv1 = bias([16])
y_conv1 = conv2d(X, W_conv1) + b_conv1
# ReLU1
relu1 = tf.nn.relu(y_conv1)
# Pool1
pool1 = max_pool(relu1)
# Conv2
W_conv2 = Weight([3, 3, 16, 32])
b_conv2 = bias([32])
y_conv2 = conv2d(pool1, W_conv2) + b_conv2
# ReLU2
relu2 = tf.nn.relu(y_conv2)
# Pool2
pool2 = max_pool(relu2)
# FC1
W_fc1 = Weight([90*53*32, 64])
b_fc1 = bias([64])
# 要把前面的網路的輸出壓扁成一維陣列
h_flat = tf.reshape(pool2, [-1, 90*53*32])
y_fc1 = tf.matmul(h_flat, W_fc1) + b_fc1
# ReLU3
relu3 = tf.nn.relu(y_fc1)
# FC2 - 輸出層
W_fc2 = Weight([64, 5]) # 分類0-9，共 10 類
b_fc2 = bias([5])

# 訓練用
y = tf.matmul(relu3, W_fc2) + b_fc2
saver = tf.train.Saver()
# 預測用
# 預測時，會先經過 softmax 計算，再取最大值為預測結果
y_pred = tf.argmax(tf.nn.softmax(y), 1)

# Cost optimizer
lossFcn = tf.nn.softmax_cross_entropy_with_logits_v2
cost = tf.reduce_mean(lossFcn(labels=y_, logits=y))

# 使用 AdamOptimizer 進行迭代
train_step = tf.train.AdamOptimizer(0.001).minimize(cost)

# 計算正確率
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.add_to_collection('input' , X)
tf.add_to_collection('output', y_pred)
#tf.add_to_collection('prob', y)

Instructions for updating:
Colocations handled automatically by placer.


# 開始進行訓練

In [7]:
# 設定輸入資料來源
img_bat, lb_bat = read_and_decode(filename, BATCH_SIZE, MAX_EPOCH)
train_x = tf.reshape(img_bat, [-1, 360, 210, 1])
train_y = tf.one_hot(lb_bat, LABEL_NUM)


with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    # 變數初始化
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    # 啟動 TFRecord 佇列
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)

    i = 0
    try:
        while not coord.should_stop():
            image_train, label_train = sess.run([train_x, train_y])
            
            sess.run(train_step, feed_dict={ X : image_train, 
                                             y_: label_train})

            if i % 1 == 0:
                train_accuracy = sess.run(accuracy, feed_dict={ X  : image_train, 
                                                                y_ : label_train})

                print('Iter %d, accuracy %4.2f%%' % (i,train_accuracy*100))
                #print (sess.run(cost))
            i += 1

    except tf.errors.OutOfRangeError:
        print('Done!')
        
    finally:
        coord.request_stop()
            
    coord.join(threads)
    
    # 存檔路徑 #
    save_path = 'C:/SPLAB/TEST1/log1/test_model'

    # 把整張計算圖存檔
    spath = saver.save(sess, save_path)
    print("Model saved in file: %s" % spath)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
Prefer Dataset.range instead.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast ins

Iter 167, accuracy 98.44%
Iter 168, accuracy 100.00%
Iter 169, accuracy 96.88%
Iter 170, accuracy 85.94%
Iter 171, accuracy 98.44%
Iter 172, accuracy 98.44%
Iter 173, accuracy 100.00%
Iter 174, accuracy 100.00%
Iter 175, accuracy 100.00%
Iter 176, accuracy 96.88%
Iter 177, accuracy 95.31%
Iter 178, accuracy 96.88%
Iter 179, accuracy 93.75%
Iter 180, accuracy 98.44%
Iter 181, accuracy 90.62%
Iter 182, accuracy 98.44%
Iter 183, accuracy 93.75%
Iter 184, accuracy 95.31%
Iter 185, accuracy 100.00%
Iter 186, accuracy 100.00%
Iter 187, accuracy 93.75%
Iter 188, accuracy 93.75%
Iter 189, accuracy 95.31%
Iter 190, accuracy 96.88%
Iter 191, accuracy 98.44%
Iter 192, accuracy 100.00%
Iter 193, accuracy 100.00%
Iter 194, accuracy 98.44%
Iter 195, accuracy 92.19%
Iter 196, accuracy 98.44%
Iter 197, accuracy 100.00%
Iter 198, accuracy 96.88%
Iter 199, accuracy 95.31%
Iter 200, accuracy 96.88%
Iter 201, accuracy 100.00%
Iter 202, accuracy 96.88%
Iter 203, accuracy 98.44%
Iter 204, accuracy 98.44%
It

# 為了要使用 混淆矩陣 而導入函式庫

In [8]:
from sklearn.metrics import confusion_matrix

# 將前面已轉為TFRecord檔的測試集進行讀取和解碼成圖片與其標籤

In [9]:
def test_decode(filename, batch_size): 

    filename_queue = tf.train.string_input_producer([filename], num_epochs = 1)
    
    # 數據讀取器
    reader = tf.TFRecordReader()
    key, values = reader.read(filename_queue)    # filename_queue
    features = tf.parse_single_example(values,
                                       features={
                                            'Label': tf.FixedLenFeature([], tf.int64),
                                            'image_raw': tf.FixedLenFeature([], tf.string),
                                       })
        # decode to tf.uint8
    image = tf.decode_raw(features['image_raw'], tf.uint8)  # tf.uint8

        # reshape
    image = tf.reshape(image, [360, 210, 1])

        # label
    label = tf.cast(features['Label'], tf.int64)    # label tf.int32
    
    image_batch, label_batch =tf.train.batch(
                                 [image, label],
                                 batch_size=batch_size,
                                 capacity=10000 + 3 * batch_size)

    return image_batch, label_batch

In [10]:
filename = 'C:/SPLAB/TEST1/test.tfrecords'

LABEL_NUM = 5

pre_img_list = []
rit_lbl_list = []

test_img, test_lbl = test_decode(filename, 1)
test_x = tf.reshape(test_img, [-1, 360, 210, 1])
test_y = tf.one_hot(test_lbl, LABEL_NUM)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.batch(batch_size)` (or `padded_batch(...)` if `dynamic_pad=True`).


# 開始進行測試，並且輸出混淆矩陣

In [11]:
with tf.Session() as sess:
    
    sess.run(tf.initialize_all_variables())
    # 變數初始化
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    # 啟動 TFRecord 佇列
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    save_path = 'C:/SPLAB/TEST1/log1/test_model.meta'
    saver = tf.train.import_meta_graph(save_path)
    saver.restore(sess, 'C:/SPLAB/TEST1/log1/test_model')
    
    x = tf.get_collection("input")[0]
    y = tf.get_collection("output")[0]

    try:
        while not coord.should_stop():
            image_test, label_test = sess.run([test_x, test_y])
            label_ = np.argmax(label_test, 1) 
            for j in np.arange(1):
                print ('label: {}'.format(label_[j]))
                rit_lbl_list.append(label_[j])
                result = sess.run(y, feed_dict = { x: image_test})             
                pre_img_list.append(result)
                print("test:")
                print(result)

    except tf.errors.OutOfRangeError:
        print('Done!')
        
    finally:
        coord.request_stop()
            
    coord.join(threads)

    C = confusion_matrix(rit_lbl_list, pre_img_list)
    print (C)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:/SPLAB/TEST1/log1/test_model
label: 3
test:
[3]
label: 4
test:
[4]
label: 0
test:
[0]
label: 3
test:
[3]
label: 3
test:
[3]
label: 2
test:
[2]
label: 0
test:
[0]
label: 1
test:
[1]
label: 0
test:
[0]
label: 2
test:
[2]
label: 3
test:
[3]
label: 0
test:
[0]
label: 3
test:
[3]
label: 2
test:
[2]
label: 2
test:
[2]
label: 2
test:
[2]
label: 0
test:
[0]
label: 4
test:
[4]
label: 3
test:
[3]
label: 1
test:
[1]
label: 3
test:
[3]
label: 1
test:
[1]
label: 1
test:
[1]
label: 2
test:
[2]
label: 2
test:
[2]
label: 1
test:
[1]
label: 2
test:
[2]
label: 1
test:
[1]
label: 3
test:
[3]
label: 3
test:
[3]
label: 4
test:
[4]
label: 0
test:
[0]
label: 0
test:
[0]
label: 2
test:
[2]
label: 4
test:
[4]
label: 4
test:
[4]
label: 3
test:
[3]
label: 0
test:
[0]
label: 2
test:
[2]
label: 1
test:
[1]
label: 4
test:
[4]
label: 4
test:
[4]
label: 3
test:
[3]
label: 1
test:
[1]
labe